### Script Purpose
- Concatenate text and NTEE codes.
- Prepare test dataset (`df_train.pkl.gz`).

In [45]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import os
import math

In [4]:
df_txt=pd.DataFrame()
file_list_txt=os.listdir('../../dataset/EIN_TXT_2014_18.pkl.gz/')
for file in file_list_txt:
    df_txt=pd.concat([df_txt,
                      pd.read_pickle('../../dataset/EIN_TXT_2014_18.pkl.gz/'+file, compression='gzip')
                     ])
df_txt.sample(3)

,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,IRS990_p3_MssnDsc,OBJECT_ID,RETURN_ID,RETURN_TYPE,SUB_DATE,TAXPAYER_NAME,TAX_PERIOD,YEAR,95_and_before
1760253,NaN,576143218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1651190,NaN,237135727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
29781,9.349332e+13,621211016,EFILE,NaN,NaN,NaN,NaN,,Highland Rim Terrace is organized as a nonprof...,The program expenses of the organization repre...,Highland Rim Terrace is organized as a nonprof...,2.014232e+17,12133297.0,990,12/29/2014,HIGHLAND RIM TERRACE INC,201406.0,2014.0,1.0


In [5]:
file_list_ntee=os.listdir('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/')
df_ntee=pd.DataFrame()
for file in file_list_ntee:
    df_ntee=pd.concat([df_ntee,
                       pd.read_pickle('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/'+file, compression='gzip')
                      ])

In [6]:
df_ein_ntee=df_ntee[['EIN', 'NTEE1']]
df_ein_ntee.drop_duplicates(inplace=True)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
df_ein_ntee.sample(3)

,EIN,NTEE1
484522,274356092,X
1809786,222501923,X
500180,300342153,T


In [62]:
df_txt_ntee_confA_no_chg=df_txt.merge(df_ein_ntee, on='EIN', how='outer')
df_txt_ntee_confA_no_chg.sample(10)

,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,IRS990_p3_MssnDsc,OBJECT_ID,RETURN_ID,RETURN_TYPE,SUB_DATE,TAXPAYER_NAME,TAX_PERIOD,YEAR,95_and_before,NTEE1
2532999,NaN,61577324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B
1603873,9.349315e+13,231685175,EFILE,NaN,NaN,NaN,NaN,,"SCHOOL FOR TODDLER, PRESCHOOL, ELEMENTARY, AND...","TODDLER, PRE-SCHOOL,ELEMENTARY, AND KINDERGART...",TO MAINTAIN A SCHOOL FOR CHILDREN WHICH PROMOT...,2.015015e+17,12660922.0,990,7/10/2015,BERKS COUNTY MONTESSORI COUNTRY DAY SCHOOL,201408.0,2015.0,1.0,B
1991576,9.349332e+13,26139581,EFILE,NaN,NaN,NaN,NaN,,PROVIDING SCHOLARSHIPS & SUPPORTING CHARITIES ...,SEE SCHEDULE ATTACHED,PROVIDING SCHOLARSHIPS,2.014032e+17,12109105.0,990,12/29/2014,JOSEPH AND ANTHONY MAIO WELLNESS FOUNDATION INC,201312.0,2014.0,0.0,B
1609802,9.349302e+13,520907015,EFILE,NaN,NaN,NaN,NaN,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,2.014102e+17,11234096.0,990,3/14/2014,INTERINDUSTRY ECONOMIC RESEARCH FUND INC,201306.0,2014.0,1.0,NaN
1778430,NaN,920117255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2016432,NaN,310977003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2478451,NaN,465190568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
2205351,9.349302e+13,251873400,EFILE,NaN,NaN,NaN,NaN,,"TO PROVIDE LOCAL, REGIONAL AND NATIONAL COMPET...",BIG CAT HOSTS 4 TO 6 SWIM MEETS PER YEAR THAT ...,"TO PROVIDE LOCAL, REGIONAL AND NATIONAL COMPET...",2.017402e+17,14559513.0,990,7/11/2017 1:48:45 PM,BIG CAT AQUATICS INC CO ETTER LAW FIRM LLC,201608.0,2017.0,0.0,N
1539661,NaN,237629895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
130345,9.349232e+13,830374553,EFILE,"During this year, the organization supported t...","Every Tribe, Inc. facilitates missionaries in ...",NaN,NaN,,NaN,NaN,NaN,2.015232e+17,13151281.0,990EZ,01/27/2016,EVERY TRIBE INC,201412.0,2016.0,0.0,NaN


In [79]:
df_txt_ntee_confA_no_chg['mission']=df_txt_ntee_confA_no_chg['IRS990_p1_ActvtyOrMssnDsc'].combine_first(df_txt_ntee_confA_no_chg['IRS990_p3_MssnDsc']).combine_first(df_txt_ntee_confA_no_chg['IRS990EZ_p3_PrmryExmptPrpsTxt'])
df_txt_ntee_confA_no_chg['prgrm_dsc']=df_txt_ntee_confA_no_chg['IRS990_p3_DscS'].fillna('')+'##'+df_txt_ntee_confA_no_chg['IRS990ScheduleO_ExplntnTxt'].fillna('')+'##'+\
                                      df_txt_ntee_confA_no_chg['IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt'].fillna('')+'##'+df_txt_ntee_confA_no_chg['IRS990ScheduleO_ExplntnTxt'].fillna('')+'##'+\
                                      df_txt_ntee_confA_no_chg['IRS990PF_p9a_DscrptnTxt'].fillna('')+'##'+df_txt_ntee_confA_no_chg['IRS990PF_p16b_RltnshpSttmntTxt'].fillna('')

In [80]:
def func_clean_str(string):
    if str(string)=='nan':
        return ''
    else:
        string_set=set([s for s in string.split('##') if s!='']) # Remove duplicates.
        return '; '.join(string_set) # Join together.

In [81]:
df_txt_ntee_confA_no_chg['mission']=df_txt_ntee_confA_no_chg['mission'].map(func_clean_str)
df_txt_ntee_confA_no_chg['prgrm_dsc']=df_txt_ntee_confA_no_chg['prgrm_dsc'].map(func_clean_str)
df_train['NTEE1']=df_train['NTEE1'].map(str.upper) # Change NTEE code to uppercase.

In [117]:
df_train=df_txt_ntee_confA_no_chg[(~df_txt_ntee_confA_no_chg['mission'].duplicated())] # Drop duplicated mission descriptions.
df_train=df_train[(~df_train['prgrm_dsc'].duplicated())] # Drop duplicated program descriptions.
df_train=df_train[(~df_train['NTEE1'].isna())] # Drop NTEE==NaN.
len(df_train)

229472

In [134]:
df_to_write=df_train
num_file=4
file_path_name='../../dataset/df_train.pkl.gz/df_train.pkl.gz'
for index in range(0, df_to_write.iloc[-1].name+1, math.ceil(df_to_write.iloc[-1].name/num_file)):
    df_temp=df_to_write.loc[index:index+math.ceil(df_to_write.iloc[-1].name/num_file)-1]
    df_temp.to_pickle(file_path_name+'-'+str(df_temp.iloc[0].name)+'-'+str(df_temp.iloc[-1].name), compression='gzip')

In [144]:
# Test reading file.
import os
file_list=os.listdir('../../dataset/df_train.pkl.gz/')
df_test=pd.DataFrame()
for file in file_list:
    df_test=pd.concat([df_test, 
                       pd.read_pickle('../../dataset/df_train.pkl.gz/'+file, compression='gzip')])
len(df_test)

229472